In [1]:
%%capture
!pip install transformers[sentencepiece]
import transformers

Processing the data.

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
%pwd

'/home/souvic/mounted/btp/vahini/Name2Demographics/Models/CBSEData/BERT'

In [4]:
%cd ../../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"
PATH_TO_FOLER = "/home/souvic/mounted/btp/vahini/Name2Demographics/"

In [6]:
%cd $PATH_TO_FOLDER

[Errno 2] No such file or directory: '$PATH_TO_FOLDER'
/home/souvic/mounted/btp/vahini/Name2Demographics


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [8]:
cbse_df = CBSEData()

In [9]:
%%capture
!pip install datasets

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


from transformers import AutoConfig
config = AutoConfig.from_pretrained("bert-base-uncased")

In [11]:
from sklearn.model_selection import train_test_split

X = list(cbse_df["Name"])
y = list(cbse_df["Gender"].values.astype('int'))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)

In [12]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [13]:
from transformers import TrainingArguments
from transformers import Trainer

In [14]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    f_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    f_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    m_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    m_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    m_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "female_precision": f_precision, "female_recall":f_recall, "female_f1": f_f1,
        "male_precision": m_precision, "male_recall": m_recall, "male_f1": m_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss,Accuracy,Female Precision,Female Recall,Female F1,Male Precision,Male Recall,Male F1
500,0.188100,0.139409,0.957462,0.967458,0.935769,0.951350,0.949923,0.974817,0.962209
1000,0.121900,0.111527,0.965287,0.966595,0.954901,0.960713,0.964263,0.973597,0.968908
1500,0.104900,0.097648,0.967583,0.967371,0.959426,0.963382,0.967750,0.974109,0.970919
2000,0.095400,0.093585,0.969178,0.968832,0.961590,0.965197,0.969452,0.975250,0.972342


TrainOutput(global_step=2135, training_loss=0.1260348934200385, metrics={'train_runtime': 8011.7091, 'train_samples_per_second': 13.323, 'train_steps_per_second': 0.266, 'total_flos': 822802742321400.0, 'train_loss': 0.1260348934200385, 'epoch': 1.0})

In [29]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Mar 17 08:02:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/

In [15]:
metrics=trainer.evaluate()
print(metrics)

{'eval_loss': 0.09358520805835724, 'eval_accuracy': 0.9691783067742147, 'eval_female_precision': 0.9688320697685943, 'eval_female_recall': 0.9615895342546599, 'eval_female_f1': 0.9651972157772623, 'eval_male_precision': 0.9694516154267386, 'eval_male_recall': 0.9752498622806327, 'eval_male_f1': 0.9723420949391919, 'eval_runtime': 515.2168, 'eval_samples_per_second': 88.792, 'eval_steps_per_second': 1.776, 'epoch': 1.0}


In [15]:
# # Load test data
# test_data = pd.read_csv("test.csv")
# X_test = list(test_data["review"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [16]:
metrics

{'eval_loss': 0.09358520805835724,
 'eval_accuracy': 0.9691783067742147,
 'eval_female_precision': 0.9688320697685943,
 'eval_female_recall': 0.9615895342546599,
 'eval_female_f1': 0.9651972157772623,
 'eval_male_precision': 0.9694516154267386,
 'eval_male_recall': 0.9752498622806327,
 'eval_male_f1': 0.9723420949391919,
 'eval_runtime': 515.2168,
 'eval_samples_per_second': 88.792,
 'eval_steps_per_second': 1.776,
 'epoch': 1.0}

In [39]:
# torch.save(model.state_dict(), "Models/CBSEData/IndicBERT/SavedModels/IndicBERT")


In [17]:
model.save_pretrained("Models/CBSEData/BERT/SavedModel/BERT")